# Notebook para manejo de dataframes y generado de informacion a partir de la API de League of Legends.

https://developer.riotgames.com/docs/lol

Autores:  
* Rubén Fernández González (rfgonzalez13@esei.uvigo.es)
* Iván Montes Blanco (imblanco17@esei.uvigo.es)
* Jacobo Cortegoso Budiño (jcbudino17@esei.uvigo.es)
* Rubén Tejo Pereira (rtpereira18@esei.uvigo.es)






El objetivo es generar un Pandas dataframe o varios a partir de los J_SON de la api del LOL. Siendo los dataframes mas importantes lo de los campeones y los de las skills

El resultado final ideal para nuestro recomendador seria un dataframe con Campeones(nombre) como filas y las siguientes columnas: Rol, Lore, Q, W, E, R, Pasiva.

Importamos las librerias de pandas y json.
Necesitamos obtener el nombre de **todos** los campeones automáticamente, por eso usamos Data Dragon, la API de League of Legends, donde tenemos un objeto J_SON con todos los campeones que usaremos como índice para después.

In [117]:
import pandas as pd
import json

df = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion.json')
df.head()

,type,format,version,data
Aatrox,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Akali', 'key': '..."
Alistar,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Alistar', 'key':..."
Amumu,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Amumu', 'key': '..."


Vemos que este DataFrame *df* contiene en su campo data el id de todos los campeones. Lo que hacemos ahora es iterar sobre el id para descargar de la API cada J_SON asociado a cada campeón y concatenarlo en un solo DataFrame.

El J_SON de cada campeón (como ejemplo mostramos a Yasuo) en Data Dragon tiene esta estructura:


In [118]:
EjemploCampeon = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion/Yasuo.json') 
EjemploCampeon = pd.read_json((EjemploCampeon['data']).to_json(), orient='index')
display(EjemploCampeon)

,id,key,name,title,image,skins,lore,blurb,allytips,enemytips,tags,partype,info,stats,spells,passive,recommended
Yasuo,Yasuo,157,Yasuo,the Unforgiven,"{'full': 'Yasuo.png', 'sprite': 'champion4.png...","[{'id': '157000', 'num': 0, 'name': 'default',...","An Ionian of deep resolve, Yasuo is an agile s...","An Ionian of deep resolve, Yasuo is an agile s...",[Dash through a minion to have Sweeping Blade ...,[Steel Tempest is very narrow. Dodge laterally...,"[Fighter, Assassin]",Flow,"{'attack': 8, 'defense': 4, 'magic': 4, 'diffi...","{'hp': 490, 'hpperlevel': 87, 'mp': 100, 'mppe...","[{'id': 'YasuoQ1Wrapper', 'name': 'Steel Tempe...","{'name': 'Way of the Wanderer', 'description':...","[{'champion': 'Yasuo', 'title': 'Beginner', 'm..."


Vemos que tenemos campos muy interesantes (y muchos otros despreciables) ya accesibles como el nombre y el lore. Sin embargo las descripciones de las skills o la pasiva estan dos niveles más abajo dentro del campo spell.

Bajamos esos niveles:

In [119]:
  s = EjemploCampeon['spells'][0]
  spellsData = [[s[0]['description'], s[1]['description'], s[2]['description'],s[3]['description'],EjemploCampeon['passive'][0]['description']]]
  EjemploHabilidades = pd.DataFrame(spellsData, columns = ['Q' , 'W' , 'E', 'R', 'Passive'], index = ['Yasuo'])
  display(EjemploHabilidades)

,Q,W,E,R,Passive
Yasuo,"Thrusts forward, damaging all enemies in a lin...",Creates a moving wall that blocks all enemy pr...,"Dashes through target enemy, dealing magic dam...",Blinks to an <factionIonia1>Airborne</factionI...,Yasuo's Critical Strike Chance is doubled. Add...


Concatenando el resultado de cada campeón y juntando los dataframes obtendremos el DataFrame *champInfo*. 

Sin embargo para el frontend de nuestra aplicacion necesitamos otros datos. Aprovechando la iteración sobre todos los campeones vamos a sacar un dataframe mas. Este dataframe contendrá el nombre de los campeones, el nombre de sus habilidades (Q,W,E,R, Passive) y la imagen de las mismas (Qimg, Wimg, Wimg, Eimg, Rimg, Passiveimg). Vamos a detallar el proceso con el Ejemplo de Yasuo:
<img src=http://ddragon.leagueoflegends.com/cdn/img/champion/splash/Yasuo_0.jpg>

In [120]:
spellsD = [[s[0]['name'], s[1]['name'], s[2]['name'],s[3]['name'], s[0]['id'], s[1]['id'], s[2]['id'], s[3]['id'], EjemploCampeon['passive'][0]['name'], EjemploCampeon['passive'][0]['image']['full']]]
EjemplofrontSpells = pd.DataFrame(spellsD, columns = ['Q' , 'W' , 'E', 'R', 'Qimg', 'Wimg', 'Eimg', 'Rimg', 'Passive', 'Passiveimg'], index = ['Yasuo'])
EjemplofrontSpells = pd.concat([EjemploCampeon['name'], EjemploCampeon['id'], EjemplofrontSpells], axis=1)
display(EjemplofrontSpells)

,name,id,Q,W,E,R,Qimg,Wimg,Eimg,Rimg,Passive,Passiveimg
Yasuo,Yasuo,Yasuo,Steel Tempest,Wind Wall,Sweeping Blade,Last Breath,YasuoQ1Wrapper,YasuoW,YasuoE,YasuoR,Way of the Wanderer,Yasuo_Passive.png


Ahora tenemos un DataFrame *EjemplofrontSpells* que contiene lo necesario para el frontend para mostrar a Yasuo en nuestra apicación web. Solo añadimos esto en un bucle para completarlo con todos los campeones. 

Las imágenes de los campeones funcionan con las siguientes rutas según su **id**:

(En ese caso el id = Yasuo)

*   Imagen Splash: http://ddragon.leagueoflegends.com/cdn/img/champion/splash/Yasuo_0.jpg
*   Imagen Loading: http://ddragon.leagueoflegends.com/cdn/img/champion/loading/Yasuo_0.jpg
*   Imagen miniatura: http://ddragon.leagueoflegends.com/cdn/10.22.1/img/champion/Yasuo.png

Las imagenes de las habilidades usan **Ximg** y la passiva **Passiveimg**

*   http://ddragon.leagueoflegends.com/cdn/10.22.1/img/spell/YasuoQ1Wrapper.png
*   http://ddragon.leagueoflegends.com/cdn/10.22.1/img/passive/Yasuo_Passive.png





Ahora solo queda generar los dos DataFrames completos con todos los campeones. Recordemos que teníamos el DataFrame *df* que básicamente contiene el 'id' de todos ellos. Iterando sobre el mismo podemos acceder a cada DataDragon de campeón y extraer la información que deseamos. El resultado debería ser *champInfo* para nuestro recomendador y *frontInfo* para el front end de nuestra aplicación.

In [121]:
champInfo = pd.DataFrame() 
frontInfo = pd.DataFrame()

for i in df['data']:
  #pasar json campeones a dataframe
  leeJson = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion/'+i['id']+'.json')
  datosChamp = pd.read_json((leeJson['data']).to_json(), orient='index')
  
  #pasar json spells y passive a dataframe
  s = datosChamp['spells'][0]
  
  #champInfo
  spellsData = [[s[0]['description'], s[1]['description'], s[2]['description'],s[3]['description'],datosChamp['passive'][0]['description']]]
  spells = pd.DataFrame(spellsData, columns = ['Q', 'W' , 'E', 'R', 'Passive'], index = [i['id']])

  #juntar dataframes y añadir a resultado
  toappend = pd.concat([datosChamp, spells], axis=1)
  champInfo = champInfo.append(toappend)

  #frontInfo
  spellsD = [[s[0]['name'], s[1]['name'], s[2]['name'],s[3]['name'], s[0]['id'], s[1]['id'], s[2]['id'], s[3]['id'], datosChamp['passive'][0]['name'], datosChamp['passive'][0]['image']['full']]]
  front = pd.DataFrame(spellsD, columns = ['Q' , 'W' , 'E', 'R', 'Qimg', 'Wimg', 'Eimg', 'Rimg', 'Passive', 'Passiveimg'], index = [i['id']])
  
  front = pd.concat([datosChamp['name'], datosChamp['id'], front], axis=1)
  frontInfo = frontInfo.append(front)
#end of loop

In [122]:
frontInfo.head()

,name,id,Q,W,E,R,Qimg,Wimg,Eimg,Rimg,Passive,Passiveimg
Aatrox,Aatrox,Aatrox,The Darkin Blade,Infernal Chains,Umbral Dash,World Ender,AatroxQ,AatroxW,AatroxE,AatroxR,Deathbringer Stance,Aatrox_Passive.png
Ahri,Ahri,Ahri,Orb of Deception,Fox-Fire,Charm,Spirit Rush,AhriOrbofDeception,AhriFoxFire,AhriSeduce,AhriTumble,Essence Theft,Ahri_SoulEater2.png
Akali,Akali,Akali,Five Point Strike,Twilight Shroud,Shuriken Flip,Perfect Execution,AkaliQ,AkaliW,AkaliE,AkaliR,Assassin's Mark,Akali_P.png
Alistar,Alistar,Alistar,Pulverize,Headbutt,Trample,Unbreakable Will,Pulverize,Headbutt,AlistarE,FerociousHowl,Triumphant Roar,Alistar_E.png
Amumu,Amumu,Amumu,Bandage Toss,Despair,Tantrum,Curse of the Sad Mummy,BandageToss,AuraofDespair,Tantrum,CurseoftheSadMummy,Cursed Touch,Amumu_Passive.png


In [123]:
champInfo.head()

,id,key,name,title,image,skins,lore,blurb,allytips,enemytips,tags,partype,info,stats,spells,passive,recommended,Q,W,E,R,Passive
Aatrox,Aatrox,266,Aatrox,the Darkin Blade,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[{'id': '266000', 'num': 0, 'name': 'default',...",Once honored defenders of Shurima against the ...,Once honored defenders of Shurima against the ...,[Use Umbral Dash while casting The Darkin Blad...,"[Aatrox's attacks are very telegraphed, so use...","[Fighter, Tank]",Blood Well,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...","[{'id': 'AatroxQ', 'name': 'The Darkin Blade',...","{'name': 'Deathbringer Stance', 'description':...","[{'champion': 'Aatrox', 'title': 'AatroxARAM',...","Aatrox slams his greatsword down, dealing phys...","Aatrox smashes the ground, dealing damage to t...","Passively, Aatrox heals when damaging enemy ch...","Aatrox unleashes his demonic form, fearing nea...","Periodically, Aatrox's next basic attack deals..."
Ahri,Ahri,103,Ahri,the Nine-Tailed Fox,"{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[{'id': '103000', 'num': 0, 'name': 'default',...",Innately connected to the latent power of Rune...,Innately connected to the latent power of Rune...,"[Use Charm to set up your combos, it will make...",[Ahri's survivability is dramatically reduced ...,"[Mage, Assassin]",Mana,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...","[{'id': 'AhriOrbofDeception', 'name': 'Orb of ...","{'name': 'Essence Theft', 'description': 'When...","[{'champion': 'Ahri', 'title': 'AhriARAM', 'ma...","Ahri sends out and pulls back her orb, dealing...",Ahri gains a brief burst of movement speed and...,Ahri blows a kiss that damages and charms an e...,"Ahri dashes forward and fires essence bolts, d...",When Ahri strikes 9 enemies with her abilities...
Akali,Akali,84,Akali,the Rogue Assassin,"{'full': 'Akali.png', 'sprite': 'champion0.png...","[{'id': '84000', 'num': 0, 'name': 'default', ...",Abandoning the Kinkou Order and her title of t...,Abandoning the Kinkou Order and her title of t...,[Akali excels at killing fragile champions. Le...,[Akali can still be hit by area effect spells ...,[Assassin],Energy,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...","[{'id': 'AkaliQ', 'name': 'Five Point Strike',...","{'name': 'Assassin's Mark', 'description': 'De...","[{'champion': 'Akali', 'title': 'AkaliARAM', '...","Akali throws out five kunai, dealing damage ba...",Akali drops a cover of smoke and briefly gains...,"Flip backward and fire a shuriken forward, dea...","Akali leaps in a direction, damaging enemies s...",Dealing spell damage to a champion creates a r...
Alistar,Alistar,12,Alistar,the Minotaur,"{'full': 'Alistar.png', 'sprite': 'champion0.p...","[{'id': '12000', 'num': 0, 'name': 'default', ...",Always a mighty warrior with a fearsome reputa...,Always a mighty warrior with a fearsome reputa...,[Using Pulverize can allow you to establish be...,[Alistar is very disruptive but very tough - t...,"[Tank, Support]",Mana,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'hp': 600, 'hpperlevel': 106, 'mp': 350, 'mpp...","[{'id': 'Pulverize', 'name': 'Pulverize', 'des...","{'name': 'Triumphant Roar', 'description': 'Al...","[{'champion': 'Alistar', 'title': 'AlistarARAM...","Alistar smashes the ground, dealing damage to ...","Alistar rams a target with his head, dealing d...","Alistar tramples nearby enemy units, ignoring ...","Alistar lets out a wild roar, removing all cro...",Alistar charges his roar by stunning or displa...
Amumu,Amumu,32,Amumu,the Sad Mummy,"{'full': 'Amumu.png', 'sprite': 'champion0.png...","[{'id': '32000', 'num': 0, 'name': 'default', ...",Legend claims that Amumu is a lonely and melan...,Legend claims that Amumu is a lonely and melan...,"[Amumu is highly dependent on teammates, so tr...",[Avoid bunching up with other allies w

Tenemos los dos DataFrames listos aunque *champInfo* contiene demasiados campos que para nuestro reconmendador son totalmente innecesarios, asi que los eliminamos:

In [124]:
del champInfo['skins']
del champInfo['key']
del champInfo['id']
del champInfo['blurb']
del champInfo['title']
del champInfo['allytips']
del champInfo['enemytips']
del champInfo['info']
del champInfo['recommended']
del champInfo['spells']
del champInfo['passive']
del champInfo['stats']
del champInfo['image']

champInfo.head()

,name,lore,tags,partype,Q,W,E,R,Passive
Aatrox,Aatrox,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,"Aatrox slams his greatsword down, dealing phys...","Aatrox smashes the ground, dealing damage to t...","Passively, Aatrox heals when damaging enemy ch...","Aatrox unleashes his demonic form, fearing nea...","Periodically, Aatrox's next basic attack deals..."
Ahri,Ahri,Innately connected to the latent power of Rune...,"[Mage, Assassin]",Mana,"Ahri sends out and pulls back her orb, dealing...",Ahri gains a brief burst of movement speed and...,Ahri blows a kiss that damages and charms an e...,"Ahri dashes forward and fires essence bolts, d...",When Ahri strikes 9 enemies with her abilities...
Akali,Akali,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,"Akali throws out five kunai, dealing damage ba...",Akali drops a cover of smoke and briefly gains...,"Flip backward and fire a shuriken forward, dea...","Akali leaps in a direction, damaging enemies s...",Dealing spell damage to a champion creates a r...
Alistar,Alistar,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,"Alistar smashes the ground, dealing damage to ...","Alistar rams a target with his head, dealing d...","Alistar tramples nearby enemy units, ignoring ...","Alistar lets out a wild roar, removing all cro...",Alistar charges his roar by stunning or displa...
Amumu,Amumu,Legend claims that Amumu is a lonely and melan...,"[Tank, Mage]",Mana,"Amumu tosses a sticky bandage at a target, stu...","Overcome by anguish, nearby enemies lose a per...",Permanently reduces the physical damage Amumu ...,Amumu entangles surrounding enemy units in ban...,Amumu's basic attacks <font color='#9b0f5f'>Cu...


**NOTA**: Para seleccionar un dato del cualquier DataFrame usaremos DataFrame.at[fila,columna].

In [141]:
print(frontInfo.at[ 'Yasuo' , 'Q' ])

Steel Tempest
